In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import tensorflow as tf

from sklearn.model_selection import train_test_split
from skimage import color, exposure, filters, io, morphology, util
from math import sqrt
from numpy import loadtxt
from tensorflow import keras
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, Reshape
from keras.optimizers import SGD

### TRANSFER LEARNING

1. Bangun sebuah model $\text{Convolutional Neural Network}$ dengan arsitektur dasar sebagai berikut :
    - Convolutional Layer ($25$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Convolutional Layer ($50$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Convolutional Layer ($100$ maps, kernel $3 \times 3$)
    - Pooling Layer ($2 \times 2$)
    - Activation Function ($\text{ReLU function}$)
    - Hidden Layer ($100$ neuron)
    - Activation Function ($\text{ReLU function}$)
    - Output Layer ($10$ kelas)
    - Activation Function ($\text{Softmax function}$)
    - Classification Result

  Dengan tambahan beberapa seperti:
  - Adam optimization
  - Early Stop

2. Implementasi model $\text{Convolutional Neural Network}$ tersebut pada dataset $\text{CIFAR 10}$ (ambil random hanya $1000$ citra per kelas) dengan inisial $\text{epoch} = 100$, $\text{batch number} = 10$, $\text{learning rate} = 0.1$, serta ratio data train dan data test sebanyak $60\% : 40\%$.

#### Mendefinisikan Fungsi Bantuan
Untuk memudahkan implementasi, berikut beberapa fungsi bantuan yang akan dipakai

##### Menghitung $\text{recall}$ atau $\text{sensitivity}$
$$\text{recall} = \frac{t_p}{t_p+t_n}$$

In [2]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

##### Menghitung $\text{precision}$
$$\text{precision} = \frac{t_p}{t_p + f_p}$$

In [3]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

##### Menghitung $\text{specificity}$
$$\text{specificity} = \frac{t_n}{t_n + f_p}$$

In [4]:
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

##### Menghitung $f_{1}\text{ Score}$
Kita tau bahwa
$$f_{\beta} = (1 + \beta^{2}) \cdot \frac{\text{precision} \cdot \text{recall}}{(\beta^{2} \cdot \text{precision}) + \text{recall}}$$
Sehingga 
$$f_{1} = (1 + 1^{2}) \cdot \frac{\text{precision} \cdot \text{recall}}{(1^{2} \cdot \text{precision}) + \text{recall}}$$
$$f_{1} = (2) \cdot \frac{\text{precision} \cdot \text{recall}}{(\text{precision}) + \text{recall}}$$



In [5]:
def f1_score(y_true, y_pred):
    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    return 2*((precision_val * recall_val)/(precision_val + recall_val + K.epsilon()))

**n.b.: Semua fungsi pada bagian penyebut ditambah epsilon untuk menghindari dibagi $0$ dikarenakan memasukkan input sebanyak $0$**

##### Fungsi Bantuan untuk Melakukan Prediksi dan Menampilkan Metric

In [6]:
def predict_and_get_metrics(model, x_test, y_test, label):
    _, accuracy, sensitivity, specificity, score_f1 = model.evaluate(x_test, y_test, verbose=0)
    return pd.DataFrame([[accuracy, sensitivity, specificity, score_f1]], columns=['Accuracy', 'Sensitivity', 'Specificity', 'F1 Score'], index=[label])

def compare_train_and_test(model, x_train, y_train, x_test, y_test, extra_label=''):
    metric_train = predict_and_get_metrics(model, x_train, y_train, 'Data Train %s' % (extra_label))
    metric_test = predict_and_get_metrics(model, x_test, y_test, 'Data Test %s' % (extra_label))
    return pd.concat([metric_train, metric_test])

##### Fungsi Bantuan untuk _Load Dataset_

In [7]:
def load_dataset_cifar10():
	(trainX, trainY), (testX, testY) = keras.datasets.cifar10.load_data()
	return np.concatenate((trainX, testX), axis=0), np.concatenate((trainY, testY), axis=0)

#### Load Dataset

In [8]:
x, y = load_dataset_cifar10()
x.shape

170500096/170498071 [==============================] - 11s 0us/step


(60000, 32, 32, 3)

#### Mengambil Dataset Sebanyak 1000 Per Kelasnya

In [9]:
idx_sample = np.array([],dtype=int)

for i in range(10):
    idx = (y == i).reshape(x.shape[0])
    idx_data = np.where(idx == True)
    sampled_list = np.random.choice(idx_data[0], size=1000, replace=False)
    idx_sample = np.concatenate((idx_sample, sampled_list), axis=0)

#### Melakukan Validasi Apakah Shape-nya Sudah Valid atau Belum

In [10]:
x_sample = x[idx_sample]
y_sample = y[idx_sample]
x_sample.shape, y_sample.shape

((10000, 32, 32, 3), (10000, 1))

#### Melakukan Normalisasi untuk $x$

In [15]:
def normalize(x):
    return x / 255.0

x_sample = normalize(x_sample)

#### Merubah Menjadi Matrix Binary untuk $y$

In [11]:
y_sample = tf.keras.utils.to_categorical(y_sample)

#### Memecah Menjadi 60 : 40 ~ Train : Test

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x_sample, y_sample, test_size=0.4)

#### Membangun Model Convolutional Neural Network

In [47]:
def create_model(learning_rate):
    convolutional_layers = []
    ## Convolutional Layers 25 map, 3 x 3, activation = relu
    convolutional_layers.append(Conv2D(25, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

    ## Pooling Layer 2 x 2
    convolutional_layers.append(MaxPooling2D(2,2))

    ## Convolutional Layers 50 map, 3 x 3, activation = relu
    convolutional_layers.append(Conv2D(50, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

    ## Pooling Layer 2 x 2
    convolutional_layers.append(MaxPooling2D(2,2))

    ## Convolutional Layers 100 map, 3 x 3, activation = relu
    convolutional_layers.append(Conv2D(100, kernel_size=(3, 3), activation=tf.nn.relu, padding='SAME', kernel_initializer=tf.keras.initializers.HeUniform()))

    ## Pooling Layer 2 x 2
    convolutional_layers.append(MaxPooling2D(2,2))

    convolutional_layers.append(Flatten())

    ## Hidden Layer 100 node
    convolutional_layers.append(Dense(100, activation=tf.nn.relu))

    ## Output 10 class
    convolutional_layers.append(Dense(10, activation=tf.nn.softmax))

    ## Menginisialisasi instance model
    convolutional_model = Sequential(convolutional_layers)

    ## Membangun model
    convolutional_model.build(input_shape=x_train.shape)

    ## Melakukan Kompilasi ke dalam Model
    convolutional_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=[
      'accuracy',
      recall,
      specificity,
      f1_score
    ])

    return convolutional_model

#### Melakukan _fitting_ dengan $\text{epochs} = 100$, $\text{batch_size} = 10$, dan $\text{learning_rate} = 0.1$, dan $\text{Early Stopping}$

In [48]:
convolutional_model = create_model(0.1)
es = EarlyStopping(monitor='val_loss')
convolutional_model.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/100
600/600 [==============================] - 15s 25ms/step - loss: 15.5747 - accuracy: 0.1032 - recall: 3.3333e-04 - specificity: 0.9995 - f1_score: 3.3333e-04 - val_loss: 2.3144 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/100
600/600 [==============================] - 14s 24ms/step - loss: 2.3237 - accuracy: 0.1038 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3170 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


3. Lakukan skenario eksperimen untuk mendapatkan metrik evaluasi optimal ($\text{akurasi}$, $\text{sensitivity}$, $\text{specifity}$, dan $f_{1} \text{ score}$) dengan mengubah parameter yaitu:
    1. Mengubah epoch, dengan $\text{learning_rate} = 0.1$ dan $\text
{batch_number} = 10$.
    2. Mengubah jumlah $\text{batch number}$ dengan $\text{epoch}$ optimal dari skenario (1) dan $\text{learning_rate} = 0.1$.
    3. Mengubah $\text{learning_rate}$ dengan $\text{epoch}$ optimal dari skenario (1) dan jumlah $\text{batch_number}$ optimal dari skenario (2).

##### Melakukan Skenario 3.1. (Mengubah $\text{epoch}$)

###### Skenario 3.1.1. ($\text{epoch} = 50$)

In [49]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=50, batch_size=10, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/50
600/600 [==============================] - 15s 25ms/step - loss: 32.3971 - accuracy: 0.0992 - recall: 5.0000e-04 - specificity: 0.9996 - f1_score: 5.8333e-04 - val_loss: 2.3097 - val_accuracy: 0.1055 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/50
600/600 [==============================] - 15s 24ms/step - loss: 2.3231 - accuracy: 0.1003 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3266 - val_accuracy: 0.1040 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Skenario 3.1.2. ($\text{epoch}$ = 25)

In [50]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=25, batch_size=10, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/25
600/600 [==============================] - 15s 24ms/step - loss: 37.5879 - accuracy: 0.1015 - recall: 8.3333e-04 - specificity: 0.9994 - f1_score: 8.8889e-04 - val_loss: 2.3227 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/25
600/600 [==============================] - 14s 24ms/step - loss: 2.3274 - accuracy: 0.0975 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3280 - val_accuracy: 0.0965 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Skenario 3.1.3. ($\text{epoch}$ = 10)

In [51]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=10, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
600/600 [==============================] - 15s 24ms/step - loss: 25.4538 - accuracy: 0.0953 - recall: 0.0012 - specificity: 0.9990 - f1_score: 0.0012 - val_loss: 2.3183 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
600/600 [==============================] - 15s 24ms/step - loss: 2.3247 - accuracy: 0.0978 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3241 - val_accuracy: 0.0972 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Kesimpulan Skenario $3.1.$

Berdasarkan hasil dari skenario $3.1.1$, $3.1.2$, dan $3.1.3$, ketiga skenario mengalami _early stop_ pada $\text{epoch} \leq 4$. Dengan demikian epoch optimal dari skenario-skenario tersebut adalah $\text{epoch}$ dari skenario $3.1.3$, yaitu $10$.

##### Melakukan Skenario $3.2.$ (Mengubah $\text{batch_number}$)

###### Skenario $3.2.1.$ ($\text{batch_number} = 25$)

In [52]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 53ms/step - loss: 63.2099 - accuracy: 0.0967 - recall: 3.3333e-04 - specificity: 0.9989 - f1_score: 4.3548e-04 - val_loss: 2.3440 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
240/240 [==============================] - 13s 52ms/step - loss: 2.3198 - accuracy: 0.0937 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3260 - val_accuracy: 0.1010 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
240/240 [==============================] - 13s 53ms/step - loss: 2.3182 - accuracy: 0.0957 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3128 - val_accuracy: 0.1040 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
240/240 [==============================] - 13s 53ms/step - loss: 2.3152 - accuracy: 0.1072 - recall: 0.0000e+00 - specificit

###### Skenario $3.2.2.$ ($\text{batch_number} = 50$)

In [53]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=50, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
120/120 [==============================] - 12s 102ms/step - loss: 64.9252 - accuracy: 0.1012 - recall: 0.0032 - specificity: 0.9969 - f1_score: 0.0033 - val_loss: 2.3124 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
120/120 [==============================] - 12s 99ms/step - loss: 2.3151 - accuracy: 0.1025 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3096 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
120/120 [==============================] - 12s 99ms/step - loss: 2.3092 - accuracy: 0.1038 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3138 - val_accuracy: 0.1010 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Skenario $3.2.3.$ ($\text{batch_number} = 100$)

In [54]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
60/60 [==============================] - 11s 191ms/step - loss: 457.3477 - accuracy: 0.1007 - recall: 0.0075 - specificity: 0.9932 - f1_score: 0.0076 - val_loss: 2.3055 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
60/60 [==============================] - 11s 183ms/step - loss: 2.3066 - accuracy: 0.1005 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3095 - val_accuracy: 0.0955 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Skenario $3.2.4.$ ($\text{batch_number} = 500$)

In [55]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=500, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
12/12 [==============================] - 10s 841ms/step - loss: 798.4647 - accuracy: 0.0987 - recall: 0.0170 - specificity: 0.9797 - f1_score: 0.0180 - val_loss: 2.3108 - val_accuracy: 0.0955 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
12/12 [==============================] - 10s 819ms/step - loss: 2.3081 - accuracy: 0.0997 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3042 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
12/12 [==============================] - 10s 818ms/step - loss: 2.3082 - accuracy: 0.1047 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3054 - val_accuracy: 0.0965 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


###### Skenario $3.2.5.$ ($\text{batch_number} = 1000$)

In [56]:
convolutional_model = create_model(0.1)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=1000, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
6/6 [==============================] - 9s 2s/step - loss: 1637.4381 - accuracy: 0.0962 - recall: 0.0473 - specificity: 0.9494 - f1_score: 0.0473 - val_loss: 2.4419 - val_accuracy: 0.0965 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
6/6 [==============================] - 9s 1s/step - loss: 2.3754 - accuracy: 0.0967 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3395 - val_accuracy: 0.1028 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
6/6 [==============================] - 9s 1s/step - loss: 2.3338 - accuracy: 0.1008 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3277 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
6/6 [==============================] - 9s 1s/step - loss: 2.3178 - accuracy: 0.1027 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 -

###### Kesimpulan Skenario $3.2.$

Berdasarkan hasil dari skenario $3.2.1$, $3.2.2$, $3.2.3$, $3.2.4$, dan $3.2.5$, kelima skenario mengalami _early stop_ pada $\text{epoch} \leq 4$. Dengan demikian $\text{batch_size}$ optimal dari skenario-skenario tersebut adalah $\text{batch_size}$ dari skenario $3.2.1$, yaitu $25$.

##### Melakukan Skenario $3.3.$ (Mengubah $\text{learning_rate}$)

##### Skenario $3.3.1.$ ($\text{learning_rate} = 0.25$)

In [57]:
convolutional_model = create_model(0.25)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 55ms/step - loss: 2822.5432 - accuracy: 0.1062 - recall: 0.0030 - specificity: 0.9975 - f1_score: 0.0030 - val_loss: 2.3322 - val_accuracy: 0.1040 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
240/240 [==============================] - 13s 52ms/step - loss: 2.3369 - accuracy: 0.1000 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3236 - val_accuracy: 0.0995 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
240/240 [==============================] - 13s 52ms/step - loss: 2.3304 - accuracy: 0.1010 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3168 - val_accuracy: 0.1040 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 4/10
240/240 [==============================] - 13s 52ms/step - loss: 2.3378 - accuracy: 0.0962 - recall: 0.0000e+00 - specificity: 1.0

##### Skenario $3.3.2.$ ($\text{learning_rate} = 0.05$)

In [58]:
convolutional_model = create_model(0.05)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 53ms/step - loss: 4.4686 - accuracy: 0.0973 - recall: 6.6667e-04 - specificity: 0.9994 - f1_score: 0.0011 - val_loss: 2.3061 - val_accuracy: 0.0960 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
240/240 [==============================] - 12s 52ms/step - loss: 2.3111 - accuracy: 0.1013 - recall: 0.0000e+00 - specificity: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.3070 - val_accuracy: 0.1020 - val_recall: 0.0000e+00 - val_specificity: 1.0000 - val_f1_score: 0.0000e+00


##### Skenario $3.3.3.$ ($\text{learning_rate} = 0.01$)

In [59]:
convolutional_model = create_model(0.01)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 53ms/step - loss: 2.2564 - accuracy: 0.2162 - recall: 0.0212 - specificity: 0.9969 - f1_score: 0.0368 - val_loss: 1.8174 - val_accuracy: 0.3305 - val_recall: 0.0840 - val_specificity: 0.9938 - val_f1_score: 0.1440
Epoch 2/10
240/240 [==============================] - 13s 52ms/step - loss: 1.8135 - accuracy: 0.3285 - recall: 0.0773 - specificity: 0.9923 - f1_score: 0.1287 - val_loss: 1.6918 - val_accuracy: 0.3790 - val_recall: 0.1385 - val_specificity: 0.9886 - val_f1_score: 0.2184
Epoch 3/10
240/240 [==============================] - 12s 52ms/step - loss: 1.6848 - accuracy: 0.3852 - recall: 0.1412 - specificity: 0.9894 - f1_score: 0.2217 - val_loss: 1.6921 - val_accuracy: 0.3778 - val_recall: 0.1528 - val_specificity: 0.9871 - val_f1_score: 0.2374


##### Skenario $3.3.4.$ ($\text{learning_rate} = 0.005$)

In [60]:
convolutional_model = create_model(0.005)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 53ms/step - loss: 2.0047 - accuracy: 0.2705 - recall: 0.0398 - specificity: 0.9953 - f1_score: 0.0677 - val_loss: 1.7832 - val_accuracy: 0.3305 - val_recall: 0.1015 - val_specificity: 0.9882 - val_f1_score: 0.1642
Epoch 2/10
240/240 [==============================] - 13s 52ms/step - loss: 1.6248 - accuracy: 0.4052 - recall: 0.1622 - specificity: 0.9893 - f1_score: 0.2509 - val_loss: 1.6142 - val_accuracy: 0.4145 - val_recall: 0.1853 - val_specificity: 0.9869 - val_f1_score: 0.2804
Epoch 3/10
240/240 [==============================] - 13s 52ms/step - loss: 1.4660 - accuracy: 0.4650 - recall: 0.2543 - specificity: 0.9863 - f1_score: 0.3633 - val_loss: 1.5383 - val_accuracy: 0.4552 - val_recall: 0.3028 - val_specificity: 0.9783 - val_f1_score: 0.4010
Epoch 4/10
240/240 [==============================] - 13s 53ms/step - loss: 1.3508 - accuracy: 0.5108 - recall: 0.3183 - specificity: 0.9850 - f1_score: 0.4333 - val_loss: 1.4329 - val

##### Skenario $3.3.5.$ ($\text{learning_rate} = 0.001$)

In [61]:
convolutional_model = create_model(0.001)
convolutional_model.fit(x_train, y_train, epochs=10, batch_size=25, validation_data=(x_test, y_test), use_multiprocessing=True, callbacks=[es])

Epoch 1/10
240/240 [==============================] - 13s 55ms/step - loss: 1.9290 - accuracy: 0.2995 - recall: 0.0450 - specificity: 0.9961 - f1_score: 0.0777 - val_loss: 1.6785 - val_accuracy: 0.3785 - val_recall: 0.0997 - val_specificity: 0.9934 - val_f1_score: 0.1680
Epoch 2/10
240/240 [==============================] - 13s 54ms/step - loss: 1.5386 - accuracy: 0.4405 - recall: 0.1965 - specificity: 0.9891 - f1_score: 0.2950 - val_loss: 1.7035 - val_accuracy: 0.4055 - val_recall: 0.2423 - val_specificity: 0.9781 - val_f1_score: 0.3333


###### Kesimpulan Skenario $3.3.$

Berdasarkan hasil dari skenario $3.3.1$, $3.3.2$, $3.3.3$, $3.3.4$, dan $3.3.5$, skenario yang menghasilkan akurasi terbaik adalah skenario $3.3.4$. Dengan demikian $\text{learning_rate}$ optimal dari skenario-skenario tersebut adalah $\text{learning}$ dari skenario $3.3.4$, yaitu $0.005$.